# Intelligent Data Analysis Project
### Matej Bebej (50%), Marian Kurcina (50%)

## Table of Contents
- Assignment
- Phase 2 - Data preprocessing

# Assignment

Oxygen saturation is a key indicator of the proper functioning of the respiratory and circulatory systems. When its value drops to a critically low level, it may indicate life-threatening conditions such as hypoxemia, respiratory failure, or severe infections. In such cases, immediate intervention is essential. Traditional monitoring is performed using pulse oximeters, which, however, can be affected by noise, motion artifacts, or may have limitations in certain clinical situations.

Modern machine learning–based approaches offer the possibility to estimate and predict critical oxygen saturation values with higher accuracy (critical oxygen saturation estimation). Models can utilize multimodal data, such as heart rate, respiratory rate, blood pressure, or sensor signals. By being trained on diverse datasets, it is possible to identify early warning signs of desaturation, filter out noise, and provide timely alerts even before oxygen saturation drops below a safe threshold.

The goal of this assignment is to become familiar with the issue of oxygen saturation monitoring, understand the contribution of artificial intelligence, and design a solution that could improve critical care and reduce risks associated with undiagnosed hypoxemia.

Each pair of students will work with an assigned dataset starting from Week 2. Your task is to predict the dependent variable “oximetry” (the predicted variable) using machine learning methods. In doing so, you will need to deal with various issues present in the data, such as inconsistent formats, missing values, outliers, and others.

The expected outcomes of the project are:

- the best-performing machine learning model, and

- a data pipeline for building it from the input data.

# Phase 2 – Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import dateparser
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer,PolynomialFeatures, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
observation = pd.read_csv("dataset/observation.csv", sep='\t')
patient = pd.read_csv("dataset/patient.csv", sep='\t')
station = pd.read_csv("dataset/station.csv", sep='\t')

In this phase, you are expected to carry out data preprocessing for machine learning. The result should be a dataset (CSV or TSV), where each observation is described by one row.
Since scikit-learn only works with numerical data, something must be done with the non-numerical data.

Ensure the preprocessing is reproducible on both the training and test datasets, so that you can repeat the process multiple times as needed (iteratively).

Because preprocessing can change the shape and characteristics of the data, you may need to perform EDA (Exploratory Data Analysis) again as necessary. These techniques will not be graded again, but document any changes in the chosen methods.
You can solve data-related issues iteratively across all phases, as needed.

## 2.1 Implementation of Data Preprocessing

### A - Train–Test Split

Split the data into training and test sets according to your predefined ratio. Continue working only with the training dataset.

In [2]:
before = len(observation)
observation = observation.drop_duplicates()
after = len(observation)
print("Removed duplicates:", before - after)

Removed duplicates: 1


In [3]:
X = observation.drop(columns=["oximetry"])
y = observation["oximetry"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("train shape:", X_train.shape)
print("train shape:", y_train.shape)
print("test shape:", X_test.shape)
print("test shape:", y_test.shape)

train shape: (9684, 22)
train shape: (9684,)
test shape: (2422, 22)
test shape: (2422,)


We split data into train and test parts, with 20% of observation data being test data () and 80% being train data ().

First we preprocess train data by: 
- enforcing schemas on each data frame
- removing latitude and longitude columns from observation since they are not medical data and have no corelation to oxymetry (as we learned in First Phase of the project - EDA)
- removing logical outliers - values which are outside of acceptable range
- removing records which have oxymetry missing 
- removing duplicates

We create a schema - expected format for each dataset and then we check if this format is present in the dataset, if some element is not same as refered in schema we try to cast the content to the correct type, if that is not possible we insert NaN.

In [4]:
station_schema = {
    'location':'string',
    'code':'string',
    'revision':'date',
    'station':'string',
    'latitude':'float',
    'longitude':'float',
}
observation_schema = {
    'SpO₂':'float',
    'HR':'float',
    'PI':'float',
    'RR':'float',
    'EtCO₂':'float',
    'FiO₂':'float',
    'PRV':'float',
    'BP':'float',
    'Skin Temperature':'float',
    'Motion/Activity index':'float',
    'PVI':'float',
    'Hb level':'float',
    'SV':'float',
    'CO':'float',
    'Blood Flow Index':'float',
    'PPG waveform features':'float',
    'Signal Quality Index':'float',
    'Respiratory effort':'float',
    'O₂ extraction ratio':'float',
    'SNR':'float',
    'oximetry':'int',
    'latitude':'float',
    'longitude':'float'
}
patient_schema = {
    'residence':'string',              
    'current_location':'string',    
    'blood_group':'string',          
    'job':'string',                 
    'mail':'string',                
    'user_id':'int',             
    'birthdate':'date',           
    'company':'string',             
    'name':'string',                
    'username':'string',            
    'ssn':'string',                 
    'registration':'date',        
    'station_ID':'int'            
}

In [5]:
class EnforceSchema(BaseEstimator, TransformerMixin):
    def __init__(self, schema=None):
        self.schema = schema

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.schema is None:
            return X

        X = X.copy()

        for col, col_type in self.schema.items():
            if col not in X.columns:
                continue
            if col_type == 'int':
                X[col] = pd.to_numeric(X[col], errors='coerce').astype('Int64')

            elif col_type == 'float':
                X[col] = pd.to_numeric(X[col], errors='coerce').astype(float)

            elif col_type == 'numeric':
                X[col] = pd.to_numeric(X[col], errors='coerce')

            elif col_type == 'date':
                X[col] = X[col].apply(
                    lambda x: dateparser.parse(str(x))
                    if pd.notnull(x) else pd.NaT
                )

            elif col_type == 'string':
                X[col] = X[col].astype(str)
                X[col] = X[col].replace('nan', np.nan)

            elif col_type == 'bool':
                X[col] = (
                    X[col]
                    .astype(str)
                    .str.lower()
                    .map({'true': True, 'false': False, '1': True, '0': False})
                )

            elif col_type == 'category':
                X[col] = X[col].astype('category')

    
        return X

In observation dataset we have expected ranges for each attribute, we check if all values are within the range, if not we push the value to the edge of the range.

In [6]:
valid_ranges = {
    'SpO₂': (95, 100),
    'HR': (60, 100),
    'PI': (0.2, 20),
    'RR': (12, 20),
    'EtCO₂': (35, 45),
    'FiO₂': (21, 100),
    'PRV': (20, 200),
    'BP': (60, 120),
    'Skin Temperature': (33, 38),
    'Motion/Activity index': None,
    'PVI': (10, 20),
    'Hb level': (12, 18),
    'SV': (60, 100),
    'CO': (4, 8),
    'Blood Flow Index': None,
    'PPG waveform features': None,
    'Signal Quality Index': (0, 100),
    'Respiratory effort': None,
    'O₂ extraction ratio': (0.2, 3),
    'SNR': (20, 40)
}

In [7]:
class EnforceValueRanges(BaseEstimator, TransformerMixin):
    def __init__(self, ranges=None):
        self.ranges = ranges

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if self.ranges is None:
            return X

        for col, valid_range in self.ranges.items():
            if col not in X.columns:
                continue
            if valid_range is None:
                continue  

            low, high = valid_range
            X[col] = pd.to_numeric(X[col], errors='coerce')

            mask = ~X[col].between(low, high, inclusive='both')
            X.loc[mask, col] = np.nan

        return X

For table station we have to parse the location column to get continent and city.

In [8]:
class ParseLocation(BaseEstimator, TransformerMixin):
    def __init__(self, column='location'):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if self.column not in X.columns:
            return X
        
        split_cols = X[self.column].astype(str).str.split('/', n=1, expand=True)
        X['continent'] = split_cols[0].str.strip()
        X['city'] = split_cols[1].str.strip()
        X = X.drop(columns=[self.column], errors='ignore')
        return X

Furthermore we create functions for other dataset transformations that we will need.

In [9]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns or []

    def fit(self, X, y=None):
        return self

    def transform(self, X,  y=None):
        if y is not None:
            return X, y
        return X

In [10]:
class RemoveDuplicates(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X,  y=None):
        if y is not None:
            combined = pd.concat([X, y.reset_index(drop=True)], axis=1)
            combined = combined.drop_duplicates()
            X_transformed = combined[X.columns]
            y_transformed = combined[y.name]
            return X_transformed, y_transformed
        else:
            X_transformed = X.drop_duplicates()
            return X_transformed

In [11]:
class DropNA(BaseEstimator, TransformerMixin):
    def __init__(self, how='any', subset=None):
        self.how = how
        self.subset = subset

    def fit(self, X, y=None):
        return self

    def transform(self, X,  y=None):
        self.mask_ = X.dropna(how=self.how, subset=self.subset).index
        X_transformed = X.loc[self.mask_].copy()
        
        if y is not None:
            y_transformed = y.loc[self.mask_].copy()
            return X_transformed, y_transformed
        return X_transformed

Lastly we define first version of pipelines. 
For observation pipeline we first enforce schema and ranges so all the values are of correct type and in valid range. Then we remove coordinates since these are not valid medical information and we don't to use them for our model. Then we remove null values and remove duplicates.
For station pipeline we first enforce schema to have all columns as correct data types, then we drop station name and revision date, because we dont want to use these coulumns and they wouldn't hold any relevant information. Then we parse the location to get continent and city of the station. Lastly we drop null values and remove duplicates. This dataset is now ready to provide us additional information about the obseravtions, if it would be needed we can remove deletion of coordinates columns and add to each observation innformation about continent, country (code) and city.
Lastly for patient dataset we only enforce schema and remove duplicates (we don't remove null values because most of the columns are partially empty), this dataset does not provide any additional information for us as we found out in 1st phase. 

In [12]:
observation_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=observation_schema)),
    ("ranges", EnforceValueRanges(ranges=valid_ranges)),
    ("drop_geo", DropColumns(columns=['latitude', 'longitude'])),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
])

station_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=station_schema)),
    ("drop_station_and_date", DropColumns(columns=['station', 'revision'])),
    ("parse_location", ParseLocation(column='location')),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
])

patient_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=patient_schema)),
    ("remove_duplicates", RemoveDuplicates()),
])

transformed_observation = observation_pipeline.fit_transform(X_train,y_train)
transformed_station = station_pipeline.fit_transform(pd.read_csv("dataset/station.csv", sep='\t'))
transformed_patient = patient_pipeline.fit_transform(pd.read_csv("dataset/patient.csv", sep='\t'))

transformed_station.head()

C:\Users\maria\AppData\Local\Temp\ipykernel_18860\1183780406.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[col] = X[col].replace('nan', np.nan)


,code,revision,station,latitude,longitude,continent,city
0,ES,2016-12-22,San Juan de Aznalfarache,37.35813,-6.03731,Europe,Madrid
1,CO,2023-07-22,Villa del Rosario,7.83389,-72.47417,America,Bogota
2,IN,2022-09-04,Rangia,26.44931,91.61356,Asia,Kolkata
3,US,2021-03-29,Rolla,37.95143,-91.77127,America,Chicago
4,DE,2023-09-12,Albstadt,48.21644,9.02596,Europe,Berlin


### B - Data Transformation

Transform the data into a format suitable for machine learning, i.e. each observation must be described by one row, and each attribute must be numeric.
Iteratively integrate preprocessing steps from Phase 1 as part of a unified process.

For observation dataset there are no string or category features, therefore we will show encoding of categorical features on station dataset. In pipeline we will still call encoding but it will have no effect. Furtermore we will only use station and observation datasets since we have relevant information there only.

In [13]:
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

observation_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=observation_schema)),
    ("ranges", EnforceValueRanges(ranges=valid_ranges)),
    ("drop_geo", DropColumns(columns=['latitude', 'longitude'])),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),  
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, [])
    ], remainder='passthrough').set_output(transform="pandas")),
])

station_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=station_schema)),
    ("drop_station_and_date", DropColumns(columns=['station', 'revision'])),
    ("parse_location", ParseLocation(column='location')),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, ['continent', 'city', 'code'])
    ], remainder='passthrough').set_output(transform="pandas")),
])





transformed_observation = observation_pipeline.fit_transform(X_train,y_train)
transformed_station = station_pipeline.fit_transform(pd.read_csv("dataset/station.csv", sep='\t'))

transformed_station.head()

,cat__continent_Africa,cat__continent_America,cat__continent_Asia,cat__continent_Atlantic,cat__continent_Australia,cat__continent_Europe,cat__continent_Indian,cat__continent_Pacific,cat__city_Abidjan,cat__city_Accra,...,cat__code_UY,cat__code_UZ,cat__code_VE,cat__code_VU,cat__code_YE,cat__code_ZA,remainder__revision,remainder__station,remainder__latitude,remainder__longitude
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2016-12-22,San Juan de Aznalfarache,37.35813,-6.03731
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2023-07-22,Villa del Rosario,7.83389,-72.47417
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2022-09-04,Rangia,26.44931,91.61356
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2021-03-29,Rolla,37.95143,-91.77127
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2023-09-12,Albstadt,48.21644,9.02596


For encoding we have put Column transformer in our pipelines, in here the transformations and scalers will be defined too. We have added remainder='passthrough' option in the column transformer to keep the columns that are not transformed in any way. After the transforms we had an issue with the step returning the numpy array which did not contain columns names, to fix this we have used .set_output(transform="pandas") option for the column transformer. This will force the step to output pandas dataframe to the next step.
For categorical columns we first replace null values with most frequent value and then we encode them. 

### C - Feature Scaling and Transformation

Transform the dataset attributes for machine learning using at least the following techniques:

- Scaling (2 techniques)
- Transformers (2 techniques)

 #### Scaling

##### StandardScaler
For each numeric column, it computes formula: $z = (x - mean)/ std$, with mean = 0 and std = 1
StandardScaler makes each column "comparable" in scale. Prevents some features being too influential simply because of their scale.

In [14]:
def get_numeric_columns(df):
    return df.select_dtypes(include=['int', 'float']).columns.tolist()

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

numeric_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    #("scaler", MinMaxScaler(feature_range=(-1, 1)))
])

observation_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=observation_schema)),
    ("ranges", EnforceValueRanges(ranges=valid_ranges)),
    ("drop_geo", DropColumns(columns=['latitude', 'longitude'])),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),  
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, []),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
])

station_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=station_schema)),
    ("drop_station_and_date", DropColumns(columns=['station', 'revision'])),
    ("parse_location", ParseLocation(column='location')),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, ['continent', 'city', 'code']),
    ], remainder='passthrough').set_output(transform="pandas")),
])




transformed_observation = observation_pipeline.fit_transform(X_train,y_train)
transformed_station = station_pipeline.fit_transform(pd.read_csv("dataset/station.csv", sep='\t'))

transformed_observation.head()

,num__SpO₂,num__HR,num__PI,num__RR,num__EtCO₂,num__FiO₂,num__PRV,num__BP,num__Skin Temperature,num__Motion/Activity index,...,num__SV,num__CO,num__Blood Flow Index,num__PPG waveform features,num__Signal Quality Index,num__Respiratory effort,num__O₂ extraction ratio,num__SNR,num__latitude,num__longitude
108,1.953852,0.032699,1.222765,0.732980,0.382418,-0.229489,0.320303,1.900541,0.891793,1.388571,...,0.530179,-0.268116,-0.754105,-0.465557,-0.706253,1.279517,-0.539533,1.003425,-1.285032,0.279441
12092,-0.991307,0.266432,0.549494,-1.610561,-0.139343,1.056711,0.443323,0.585270,-0.307794,-0.968697,...,0.206366,-0.165222,0.820335,0.229577,-0.751795,0.979515,0.789101,-1.425010,-0.544045,0.898907
9658,-2.102740,0.593181,0.226362,-3.443886,-0.014848,1.979283,-0.563047,-0.775902,-0.102475,2.708076,...,-0.398170,0.088075,-0.844645,-0.020597,1.879169,1.608523,0.392014,-1.071256,-0.998730,1.252600
2929,-0.522682,-0.408683,-0.316020,-0.381465,0.273402,0.226677,-0.691574,-0.831748,-2.301060,-1.472041,...,0.810748,-0.429002,-1.756642,-0.921236,0.397168,-0.562005,1.130631,0.916572,-1.540835,-0.609127
10405,0.634981,0.591865,0.352991,0.862953,1.823320,-0.107193,0.331392,-1.149611,-1.598345,-0.775176,...,1.276494,0.048805,0.019867,0.402615,0.990483,0.075276,1.134043,1.293153,-0.095177,1.111728


##### MinMax Scaler
Maps the minimum of each feature to −1 and the maximum to 1. It uses following formula: $z = -1 + 2 * ((x - x(min) / (x(max) - x(min))$
Preserves the original shape of the distribution, only makes the values more tightly bounded.
Good when we need values from smaller ranges (here from -1 to 1).

In [15]:
def get_numeric_columns(df):
    return df.select_dtypes(include=['int', 'float']).columns.tolist()

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

numeric_pipeline = Pipeline([
    #("scaler", StandardScaler()),
    ("scaler", MinMaxScaler(feature_range=(-1, 1)))
])

observation_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=observation_schema)),
    ("ranges", EnforceValueRanges(ranges=valid_ranges)),
    ("drop_geo", DropColumns(columns=['latitude', 'longitude'])),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),  
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, []),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
])

station_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=station_schema)),
    ("drop_station_and_date", DropColumns(columns=['station', 'revision'])),
    ("parse_location", ParseLocation(column='location')),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, ['continent', 'city', 'code']),
    ], remainder='passthrough').set_output(transform="pandas")),
])




transformed_observation = observation_pipeline.fit_transform(X_train,y_train)
transformed_station = station_pipeline.fit_transform(pd.read_csv("dataset/station.csv", sep='\t'))

transformed_observation.head()

,num__SpO₂,num__HR,num__PI,num__RR,num__EtCO₂,num__FiO₂,num__PRV,num__BP,num__Skin Temperature,num__Motion/Activity index,...,num__SV,num__CO,num__Blood Flow Index,num__PPG waveform features,num__Signal Quality Index,num__Respiratory effort,num__O₂ extraction ratio,num__SNR,num__latitude,num__longitude
108,0.756534,-0.086035,0.385460,0.344547,0.046418,-0.025558,0.050369,0.542354,0.302071,0.275965,...,0.448987,-0.978535,-0.192627,-0.083837,-0.217594,0.468981,-0.319556,0.574116,-0.124499,0.150960
12092,-0.248824,-0.030191,0.143535,-0.409846,-0.080694,0.306877,0.083884,0.219843,0.010913,-0.347967,...,0.392417,-0.973574,0.213642,0.081514,-0.229690,0.397657,0.448137,-0.825119,0.168459,0.431359
9658,-0.628222,0.047877,0.027425,-1.000000,-0.050365,0.545327,-0.190286,-0.113924,0.060747,0.625218,...,0.286804,-0.961361,-0.215990,0.022005,0.469048,0.547200,0.218698,-0.621290,-0.011306,0.591456
2929,-0.088854,-0.191492,-0.167469,-0.014196,0.019859,0.092344,-0.225301,-0.127617,-0.472883,-0.481194,...,0.498002,-0.986293,-0.451321,-0.192228,0.075455,0.031171,0.645475,0.524072,-0.225633,-0.251246
10405,0.306325,0.047562,0.072926,0.386386,0.397450,0.006051,0.053390,-0.205559,-0.302323,-0.296745,...,0.579367,-0.963255,0.007089,0.122674,0.233029,0.182680,0.647446,0.741054,0.345925,0.527691


We have put both scalers into our pipeline, but one will be commented out. In need we can use any of the scalers easily.

#### Transformers

##### PowerTransformer (Yeo-Johnson method)

Makes given data from skewed and non-normal into more Gaussian like shape.
There is a big chance of model behaving better when the inputs we feed him are closer to normal

In [16]:
def get_numeric_columns(df):
    return df.select_dtypes(include=['int', 'float']).columns.tolist()

numeric_pipeline = Pipeline([
    ("power", PowerTransformer(method="yeo-johnson")),
    #("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler()),
    #("scaler", MinMaxScaler(feature_range=(-1, 1)))
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

observation_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=observation_schema)),
    ("ranges", EnforceValueRanges(ranges=valid_ranges)),
    ("drop_geo", DropColumns(columns=['latitude', 'longitude'])),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),  
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, []),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
])

station_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=station_schema)),
    ("drop_station_and_date", DropColumns(columns=['station', 'revision'])),
    ("parse_location", ParseLocation(column='location')),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, ['continent', 'city', 'code']),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
])



transformed_observation = observation_pipeline.fit_transform(X_train,y_train)
transformed_station = station_pipeline.fit_transform(pd.read_csv("dataset/station.csv", sep='\t'))

transformed_observation.head()

,num__SpO₂,num__HR,num__PI,num__RR,num__EtCO₂,num__FiO₂,num__PRV,num__BP,num__Skin Temperature,num__Motion/Activity index,...,num__SV,num__CO,num__Blood Flow Index,num__PPG waveform features,num__Signal Quality Index,num__Respiratory effort,num__O₂ extraction ratio,num__SNR,num__latitude,num__longitude
108,2.241529,0.035210,1.186962,0.715950,0.376513,-0.229246,0.321797,1.904921,0.892083,1.385230,...,0.524744,-0.225364,-0.755669,-0.466986,-0.704109,1.284423,-0.532905,1.001671,-1.453282,0.244643
12092,-1.009404,0.268736,0.608312,-1.543483,-0.146314,1.056657,0.444607,0.584205,-0.306466,-0.968297,...,0.198872,-0.039309,0.819897,0.228158,-0.749841,0.980227,0.792622,-1.450753,-0.762531,0.902748
9658,-1.890576,0.594747,0.309697,-2.918234,-0.021913,1.978583,-0.561582,-0.776612,-0.101027,2.687207,...,-0.405067,0.332801,-0.845873,-0.022195,1.871305,1.619050,0.399880,-1.073147,-1.229393,1.282138
2929,-0.588099,-0.406534,-0.229970,-0.431378,0.266957,0.226903,-0.690361,-0.832310,-2.307414,-1.476076,...,0.808427,-0.597474,-1.751408,-0.921782,0.399576,-0.567079,1.128137,0.918762,-1.588683,-0.631504
10405,0.597934,0.593434,0.428541,0.858602,1.840109,-0.106945,0.332869,-1.149125,-1.600628,-0.773598,...,1.282045,0.332801,0.017320,0.401428,0.990062,0.069165,1.131478,1.276149,-0.247202,1.130796


##### Polynomial features

We create new features which are results of us multiplying existing ones and by that expands the feature space.
We are hoping that this will result in boosting accuracy of our model.

In [17]:
def get_numeric_columns(df):
    return df.select_dtypes(include=['int', 'float']).columns.tolist()

numeric_pipeline = Pipeline([
    #("power", PowerTransformer(method="yeo-johnson")),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler()),
    #("scaler", MinMaxScaler(feature_range=(-1, 1)))
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

observation_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=observation_schema)),
    ("ranges", EnforceValueRanges(ranges=valid_ranges)),
    ("drop_geo", DropColumns(columns=['latitude', 'longitude'])),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),  
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, []),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
])

station_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=station_schema)),
    ("drop_station_and_date", DropColumns(columns=['station', 'revision'])),
    ("parse_location", ParseLocation(column='location')),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, ['continent', 'city', 'code']),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
])


transformed_observation = observation_pipeline.fit_transform(X_train,y_train)
transformed_station = station_pipeline.fit_transform(pd.read_csv("dataset/station.csv", sep='\t'))

transformed_observation.head()

,num__SpO₂,num__HR,num__PI,num__RR,num__EtCO₂,num__FiO₂,num__PRV,num__BP,num__Skin Temperature,num__Motion/Activity index,...,num__O₂ extraction ratio^2,num__O₂ extraction ratio SNR,num__O₂ extraction ratio latitude,num__O₂ extraction ratio longitude,num__SNR^2,num__SNR latitude,num__SNR longitude,num__latitude^2,num__latitude longitude,num__longitude^2
108,1.953852,0.032699,1.222765,0.732980,0.382418,-0.229489,0.320303,1.900541,0.891793,1.388571,...,-0.579083,0.527485,-1.259206,0.252091,1.000101,-1.245683,0.353315,-1.401214,-0.070430,-0.788591
12092,-0.991307,0.266432,0.549494,-1.610561,-0.139343,1.056711,0.443323,0.585270,-0.307794,-0.968697,...,0.765317,-0.922922,-0.475862,0.987467,-1.320641,-0.693880,0.604919,-1.155540,0.461603,0.046585
9658,-2.102740,0.593181,0.226362,-3.443886,-0.014848,1.979283,-0.563047,-0.775902,-0.102475,2.708076,...,0.342165,-0.755059,-0.973374,1.307817,-1.052870,-0.996725,0.949592,-1.381002,0.165893,0.844020
2929,-0.522682,-0.408683,-0.316020,-0.381465,0.273402,0.226677,-0.691574,-0.831748,-2.301060,-1.472041,...,1.143825,1.468764,-1.559376,-0.667294,0.897621,-1.532609,-0.679436,-1.339685,0.073831,-0.738530
10405,0.634981,0.591865,0.352991,0.862953,1.823320,-0.107193,0.331392,-1.149611,-1.598345,-0.775176,...,1.147674,1.835078,0.049683,1.267830,1.352410,0.173206,1.396111,-0.700193,0.982122,0.498488


### D - Justification and Documentation

We transformed the observation dataset to make it compatible with model training by ensuring it contains only numerical values. We also demonstrated two different methods for data transformation and scaling, which help the model learn more effectively. For each method we described how the process works and what this hopes to achieve. In 3rd phase we can use these methods to make our models more accurate.

## 2.2 Feature Selection 

### A - Identification of Informative Features

Identify which attributes (features) in your data are informative with respect to the target variable (use at least 3 techniques and compare their results).

#### Variance Threshold

Removes features that have a low variance, we will use this technique before scaling, because scaling and transforming can change the variance.

In [18]:
def get_numeric_columns(df):
    return df.select_dtypes(include=['int', 'float']).columns.tolist()

numeric_pipeline = Pipeline([
    #("power", PowerTransformer(method="yeo-johnson")),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler()),
    #("scaler", MinMaxScaler(feature_range=(-1, 1)))
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

observation_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=observation_schema)),
    ("ranges", EnforceValueRanges(ranges=valid_ranges)),
    ("drop_geo", DropColumns(columns=['latitude', 'longitude'])),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),  
    ("variance_threshold", VarianceThreshold(threshold=0.01).set_output(transform="pandas")),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, []),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),

])

station_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=station_schema)),
    ("drop_station_and_date", DropColumns(columns=['station', 'revision'])),
    ("parse_location", ParseLocation(column='location')),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, ['continent', 'city', 'code']),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
])




transformed_observation = observation_pipeline.fit_transform(X_train, y_train)
transformed_station = station_pipeline.fit_transform(pd.read_csv("dataset/station.csv", sep='\t'))

select = VarianceThreshold(threshold=0.01)
select.fit(X_train)
select_features = X_train.columns[select.get_support()]

mask = select.get_support()

removed_features = X_train.columns[~mask]

print("Removed features due to low variance:")
print(removed_features.tolist())

print("Selected:\n", select_features.tolist())

Removed features due to low variance:
['CO', 'O₂ extraction ratio']
Selected:
 ['SpO₂', 'HR', 'PI', 'RR', 'EtCO₂', 'FiO₂', 'PRV', 'BP', 'Skin Temperature', 'Motion/Activity index', 'PVI', 'Hb level', 'SV', 'Blood Flow Index', 'PPG waveform features', 'Signal Quality Index', 'Respiratory effort', 'SNR', 'latitude', 'longitude']


#### Mutual Information

Measures the dependency between the variables, higher the values, higher the dependency.

In [36]:
def get_numeric_columns(df):
    return df.select_dtypes(include=['int', 'float']).columns.tolist()

numeric_pipeline = Pipeline([
    #("power", PowerTransformer(method="yeo-johnson")),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler()),
    #("scaler", MinMaxScaler(feature_range=(-1, 1)))
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

observation_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=observation_schema)),
    ("ranges", EnforceValueRanges(ranges=valid_ranges)),
    ("drop_geo", DropColumns(columns=['latitude', 'longitude'])),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),  
    ("variance_threshold", VarianceThreshold(threshold=0.01).set_output(transform="pandas")),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, []),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
    ("select_kbest", SelectKBest(score_func=f_regression, k=15).set_output(transform="pandas")) 

])

station_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=station_schema)),
    ("drop_station_and_date", DropColumns(columns=['station', 'revision'])),
    ("parse_location", ParseLocation(column='location')),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, ['continent', 'city', 'code']),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
])




transformed_observation = observation_pipeline.fit_transform(X_train, y_train)
transformed_station = station_pipeline.fit_transform(pd.read_csv("dataset/station.csv", sep='\t'))

select_kbest_step = observation_pipeline.named_steps['select_kbest']
mask = select_kbest_step.get_support() 
column_transformer = observation_pipeline.named_steps['encode']
feature_names = column_transformer.get_feature_names_out()
selected_features = feature_names[mask]
print(selected_features)

['num__SpO₂' 'num__RR' 'num__EtCO₂' 'num__SpO₂^2' 'num__SpO₂ RR'
 'num__SpO₂ EtCO₂' 'num__HR RR' 'num__RR^2' 'num__RR EtCO₂' 'num__RR BP'
 'num__RR Skin Temperature' 'num__RR Motion/Activity index' 'num__RR PVI'
 'num__RR Hb level' 'num__RR SV']


#### Recursive feature elimination

In [ ]:
def get_numeric_columns(df):
    return df.select_dtypes(include=['int', 'float']).columns.tolist()

numeric_pipeline = Pipeline([
    #("power", PowerTransformer(method="yeo-johnson")),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler()),
    #("scaler", MinMaxScaler(feature_range=(-1, 1)))
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

observation_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=observation_schema)),
    ("ranges", EnforceValueRanges(ranges=valid_ranges)),
    ("drop_geo", DropColumns(columns=['latitude', 'longitude'])),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),  
    ("variance_threshold", VarianceThreshold(threshold=0.01).set_output(transform="pandas")),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, []),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
    ("select_kbest", SelectKBest(score_func=f_regression, k=15).set_output(transform="pandas")) 

])

station_pipeline = Pipeline([
    ("schema", EnforceSchema(schema=station_schema)),
    ("drop_station_and_date", DropColumns(columns=['station', 'revision'])),
    ("parse_location", ParseLocation(column='location')),
    ("drop_na", DropNA(how='any')),
    ("remove_duplicates", RemoveDuplicates()),
    ("encode", ColumnTransformer([
        ('cat', cat_pipeline, ['continent', 'city', 'code']),
        ('num', numeric_pipeline, get_numeric_columns)
    ], remainder='passthrough').set_output(transform="pandas")),
])




transformed_observation = observation_pipeline.fit_transform(X_train, y_train)
transformed_station = station_pipeline.fit_transform(pd.read_csv("dataset/station.csv", sep='\t'))

select_kbest_step = observation_pipeline.named_steps['select_kbest']
mask = select_kbest_step.get_support() 
column_transformer = observation_pipeline.named_steps['encode']
feature_names = column_transformer.get_feature_names_out()
selected_features = feature_names[mask]
print(selected_features)

### B - Ranking of Features

Rank the identified features by importance.

### C - Justification and Documentation

Justify your choices/decisions for implementation (i.e., provide documentation).

## 2.3 Reproducibility of Preprocessing

### Code Generalization for Reuse and Pipeline Implementation

Modify your preprocessing code for the training dataset so that it can be reused without further modifications to preprocess the test dataset in a machine learning context. Use the sklearn.pipeline functionality.